In [1]:
import pandas as pd

### 서울대병원 데이터 크롤링 [(링크)](http://www.snuh.org/health/nMedInfo/nList.do?searchNWord=&sortType=A)

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time


def snu_crowling(disease, idx, df):
    chrome_option = webdriver.ChromeOptions()
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_option)
    
    # 서울대학교 병원 홈페이지
    driver.get('http://www.snuh.org/health/nMedInfo/nList.do?searchNWord=&sortType=A')

    # 검색 버튼 클릭
    search_btn = driver.find_element(By.XPATH, '//*[@id="gnb"]/button[2]')
    search_btn.click()
    time.sleep(1)

    # 검색창에 질병명 입력 및 전송
    search_ipt = driver.find_element(By.XPATH, '//*[@id="totSearchInp"]')
    search_ipt.send_keys(disease)
    search_ipt.send_keys(Keys.RETURN)
    time.sleep(1)

    # 새로 열린 검색 결과 탭으로 이동
    driver.switch_to.window(driver.window_handles[-1])

    # 검색 결과 페이지의 내비게이션 바에서 "건강정보" 클릭
    navi_btn = driver.find_element(By.XPATH, '//*[@id="gnb"]/ul/li[4]/a')
    navi_btn.click()
    time.sleep(1)
    
    more_btn = driver.find_element(By.XPATH, '//*[@id="content"]/div[3]/a')
    more_btn.click()
    time.sleep(1)
    
    sort_acc = driver.find_element(By.XPATH, '//*[@id="rank"]')
    sort_acc.click()
    time.sleep(1)
    
    results_list = driver.find_element(By.CLASS_NAME, 'thumbType02').find_elements(By.CLASS_NAME, 'item')
    elem_list = []
    title_list = []
    for result in results_list:
        elem_list.append(result.find_element(By.TAG_NAME, 'strong'))
        title_list.append(result.find_element(By.TAG_NAME, 'strong').text.replace(" ", ""))
    
    best = []
    for title in title_list:
        if '[N의학정보]' + disease.replace(" ", "") in title:
            best.insert(0, title_list.index(title))
        elif '[N의학정보]' in title and disease.replace(" ", "") in title:
            best.append(title_list.index(title))
        
    elem_list[best[0]].click()
        
    # # 건강정보 결과 중 질병 탐색 과정
    # for i in range(1,10):
    #     disease_btn = driver.find_element(By.XPATH, f'//*[@id="content"]/div[3]/div[{i}]/div/div[1]/a/strong')
                                                        
    #     # 띄어쓰기로 인해 검색되지 않는 경우를 위하여 띄어쓰기 제거
    #     searched_disease = disease_btn.text.replace(" ", "")
    #     # 건강정보 결과 중 "N의학정보"가 포함되어 있는 경우 
    #     if '[N의학정보]' + disease.replace(" ", "") in searched_disease:
    #         disease_btn.click()
    #         break
    #     elif '[N의학정보]' in searched_disease and disease.replace(" ", "") in searched_disease:
    #         disease_btn.click()
    #         # 타겟 질병명이 결과에 포함되는 경우를 클릭 후 반복문 종료
    #         break
    #     else:
    #         continue

    #         disease_btn.click()
    #         time.sleep(1)
    #         break

    # 새로 열린 질환 세부 정보 페이지로 이동
    driver.switch_to.window(driver.window_handles[-1])

    xpath_dict = {
        "searched_name": '//*[@id="content"]/div[2]/div[1]/h3',                   # 검색된 질환명
        "symptom_keys": '//*[@id="content"]/div[2]/div[2]/div[4]/p',              # 증상 키워드
        "define_text": '//*[@id="content"]/div[2]/div[3]/div[2]/div[1]/p',        # 질환 정의
        "symptom_text": '//*[@id="content"]/div[2]/div[3]/div[2]/div[2]/p',       # 증상
        "cause_text": '//*[@id="content"]/div[2]/div[3]/div[2]/div[3]/p',         # 원인
        "related_organs": '//*[@id="content"]/div[2]/div[3]/div[2]/div[4]/p',     # 관련 신체기관
        "diagnosis_text": '//*[@id="content"]/div[2]/div[3]/div[2]/div[5]/p',     # 진단
        "treat_text": '//*[@id="content"]/div[2]/div[3]/div[2]/div[7]/p',         # 치료
    }
    
    cols = ['질환명', '영문명', '증상 키워드', '정의', '증상', '원인', '관련신체기관', '진단', '치료', 'URL']

    # 질환 세부 정보 수집
    err_list = []
    for info, xpath in xpath_dict.items():
        # 질환명은 따로 수집하여, 파싱을 통해 국문명과 영문명을 추출
        if info == 'searched_name':
            info = driver.find_element(By.XPATH, xpath).text
            kr_name = info.split('\n')[0]
            en_name = info.split('\n')[1].replace('[','').replace(']','')   
            df.loc[idx, '질환명'] = kr_name
            df.loc[idx, '영문명'] = en_name
        else: 
            # 예외 처리
            try:
                info = driver.find_element(By.XPATH, xpath).text
            except:
                err_list.append(info)
                info = None
                continue
            df.loc[idx, cols[list(xpath_dict.values()).index(xpath)+1]] = info
    
    df.loc[idx, 'URL'] = driver.current_url
    
    if len(err_list) != 0:
        print(f"Error: '{disease}'의 '{err_list}' 정보 수집 실패")
    else:
        print(f"'{disease}'의 정보 수집 완료")
    driver.close()

In [9]:
a = str(diseases)
a = a.replace('[','').replace(']','').replace('\'','')
a

'과민성 대장 증후군, 당뇨병, 심부전, 천식, 급성 대장염, 감기, 급성 경막하 출혈, 인플루엔자, 긴장성 두통, 고산병, 아나필락시스, 루게릭병, 장폐색, 아토피 피부염, 대상포진, 돌발성 난청, 만성 비염, 급성 신우신염, 뇌전증, 패혈증'

In [3]:
import pandas as pd
# 질환 20개 리스트
diseases = ['과민성 대장 증후군', '당뇨병', '심부전', '천식', '급성대장염',
           '감기', '급성 경막하 출혈', '인플루엔자', '긴장성 두통', '고산병',
           '아나필락시스', '루게릭병', '장폐색', '아토피 피부염', '대상포진',
           '돌발성 난청', '만성 비염', '신우신염', '뇌전증', '패혈증']

snu_df = pd.DataFrame(columns=['질환명', '영문명', '증상 키워드', '정의', '증상', '원인', '관련신체기관', '진단', '치료', "URL"])

for idx in range(len(diseases)):
    disease = diseases[idx]
    snu_crowling(disease, idx, snu_df)

'과민성 대장 증후군'의 정보 수집 완료
'당뇨병'의 정보 수집 완료
'심부전'의 정보 수집 완료
'천식'의 정보 수집 완료
'급성대장염'의 정보 수집 완료
'감기'의 정보 수집 완료
'급성 경막하 출혈'의 정보 수집 완료
'인플루엔자'의 정보 수집 완료
'긴장성 두통'의 정보 수집 완료
'고산병'의 정보 수집 완료
'아나필락시스'의 정보 수집 완료
'루게릭병'의 정보 수집 완료
'장폐색'의 정보 수집 완료
'아토피 피부염'의 정보 수집 완료
'대상포진'의 정보 수집 완료
'돌발성 난청'의 정보 수집 완료
'만성 비염'의 정보 수집 완료
'신우신염'의 정보 수집 완료
'뇌전증'의 정보 수집 완료
'패혈증'의 정보 수집 완료


In [4]:
snu_df.to_csv('snuh_crowling.csv', index=False, encoding='utf-8-sig')
snu_df

,질환명,영문명,증상 키워드,정의,증상,원인,관련신체기관,진단,치료,URL
0,과민성 대장 증후군,irritable bowel syndrome,"두통 , 복통 , 위경련 , 배 속 가스 , 변비 , 설사",과민성 대장 증후군은 대장 내시경이나 엑스선검사로 확인되는 특정 질환은 없지만 식사...,전형적인 증상으로는 복통과 배변 습관의 변화를 들 수 있는데 복통이 심하더라도 배변...,"명확한 원인은 아직 밝혀진 것이 없으며, 내장 감각의 과민성 증가, 위장관 운동성의...",대장,"과민성 대장 증후군은 생화학적, 구조적 이상으로 설명할 수 없으므로 어떤 한 가지 ...",치료는 과민성 대장 증후군의 원인이 되는 심리적 불안과 갈등을 제거하는 것이 가장 ...,http://www.snuh.org/health/nMedInfo/nView.do?c...
1,당뇨병,diabetes mellitus,"무감각증 , 소양감 , 변비 , 저혈압 , 거품뇨 , 단백뇨",당뇨병은 인슐린의 분비량이 부족하거나 정상적인 기능이 이루어지지 않는 등의 대사질환...,약한 고혈당에서는 대부분의 환자들이 증상을 느끼지 못하거나 모호해서 당뇨병이라고 생...,"당뇨병은 제1형과 제2형으로 구분되는데, 제1형 당뇨병은 이전에 '소아 당뇨병'이라...",췌장,혈액검사로 진단한다. 증상이 없는 경우 8시간 이상 금식 후에 측정한 혈당이 126...,제1형 당뇨병의 경우에는 인슐린 치료가 필요하다. 제2형 당뇨병의 경우에는 생활 습...,http://www.snuh.org/health/nMedInfo/nView.do?c...
2,심부전,heart failure,"기침 , 피로 , 호흡장애 , 부종",심부전이란 심장의 구조적 또는 기능적 이상으로 인해 심장이 혈액을 받아들이는 충만 ...,가장 흔하고 중요한 증상은 숨이 차는 것(호흡곤란)이다. 계단을 오르기만 해도 과도...,"매우 다양한 원인에 의해 심부전이 초래할 수 있는데, 심장 혈관(관상동맥) 질환(예...",심장,"호흡곤란, 피로감 등과 같은 심부전의 주관적 증상과 진찰 및 청진 시 이상 징후, ...",급성이냐 만성이냐에 따라 치료 방침은 다르다. 급성 심부전은 주로 1주일 이내 갑자...,http://www.snuh.org/health/nMedInfo/nView.do?c...
3,천식,asthma,"호흡장애 , 만성 기침 , 기침","천식이란 폐 속에 있는 기관지가 아주 예민해진 상태로, 때때로 기관지가 좁아져서 숨...","기관지 천식의 대표적인 증상은 호흡곤란, 기침, 천명(쌕쌕거리는 거친 숨소리)이다....",천식은 유전적 요인과 환경적 요인이 합쳐져서 생기는 대표적인 알레르기 질환이다.\n...,기관지,천식은 특징적인 증상과 진찰 소견 및 여러 가지 검사로 진단된다.\n기관지 천식의 ...,기관지 천식은 만성적이고 재발이 많은 질환이다. 따라서 증상을 잘 조절하고 폐기능을...,http://www.snuh.org/health/nMedInfo/nView.do?c...
4,급성 대장염,acute colitis,"구역 , 발열 , 혈변 , 구토 , 설사 , 복통","급성 장염은 일반적으로 급성 감염성 장염을 일컫는데 다양한 종류의 세균, 바이러스,...",대표적인 증상은 설사이다. 세균에 감염되면 장관 내로 수분과 전해질이 흡수되지 않고...,"급성 장염을 일으키는 원인은 다양한 종류의 세균, 바이러스, 원충으로 이들에 오염된...","위, 소장, 대장",전형적인 증상과 병력을 통해 임상적으로 진단한다. 확진을 위해서 혈액 또는 대변 배...,성인에게 나타나는 급성 설사는 대부분 입으로 충분한 수분을 섭취하는 것만으로도 치료...,http://www.snuh.org/health/nMedInfo/nView.do?c...
5,감기,common cold,"기침 , 인두통 및 인후통 , 비루 , 비폐색 , 재채기 , 근육통 , 발열","감기는 바이러스에 의해 코와 목 부분을 포함한 상부 호흡기계의 감염 증상으로, 사람...",감기 바이러스에 노출된 지 1~3일 후에 증상이 나타난다. 증상은 감기 바이러스가 ...,200여개 이상의 서로 다른 종류의 바이러스가 감기를 일으킨다. 그 중 30~50%...,"상부 호흡기계, 코, 목",병력 조사와 함께 임상 증상을 관찰하여 진단할 수 있다. 감기의 경우 확진 검사는 ...,특이적인 치료법은 없다.\n세균에 의한 2차 감염을 방지할 목적으로 항생제를 일률적...,http://www.snuh.org/health/nMedInfo/nView.do?c...
6,급성 경막하 출혈,acute subdural hemorrhage,"두통 , 호흡장애 , 혼수 , 구역 , 구토",급성 경막하 출혈은 뇌를 둘러싸고 있는 경막(dura)이라는 막의 안쪽에서 뇌 표면...,급성 경막하 출혈은 가장 심각한 형태의 두부 외상으로서 많은 경우에 의식장애가 동반...,노인의 경우 낙상(落傷)에 의하여 주로 발생하며 젊은이들에서는 교통사고와 폭행에 의...,뇌,젊은 사람이 오토바이 사고를 당하고 의식을 잃었거나 노인이 길 또는 계단에 쓰러져 ...,환자의 의식이 저하되고 있으며 전산화 단층촬영에서 출혈량이 상당하여 뇌가 압박을 받...,http://www.snuh.org/health/nMedInfo/nView.do?c...
7,인플루엔자,influenza,"기침 , 근육통 , 두통 , 인두통 및 인후통 , 발열","독감은 인플루엔자 바이러스에 의한 급성 호흡기 질환이다. 독감은 상부 호흡기계(코,...","두통, 발열, 오한, 근육통과 같은 전신 증상이 갑자기 발생하면서 목이 아프고 기침...",독감 바이러스 또는 인플루엔자 바이러스(influenza virus)가 원인 병원체...,"기도, 심장, 뇌수막",독감 유행 시기에 앞서 독감 증상이 있는 경우에 독감을 의심해 볼 수 있다. 호흡기...,독감 바이러스에 대한 특이적인 치료제가 있다. 그러나 증상 발생 48시간 안에 사용...,http://www.snuh.org/health/nMedInfo/nView.do?c...
8,긴장성 두통,tension type headache,두통,긴장성두통은 대부분의 사람들이 경험할 정도의 가장 흔한 두통이다. 일반적으로 스트레...,긴장성두통은 흔히 두피(두개골 밖)에 분포하는 근육이 지속적으로 수축하면서 발생한다...,긴장성두통의 원인에 대해서는 명확히 밝혀진 것이 없다. 긴장성두통 환자에게서 두개(...,머리(두피),국제두통협회에서 긴장성두통의 진단 기준을 발표하였고 이 기준이 임상에서 널리 쓰이고...,긴장형두통은 만성으로 변형되지 않는 한 대개 치료에 잘 반응하며 장기적인 합병증이나...,http://www.snuh.org/health/nMedInfo/nView.do?c...
9,고산병,high-altitude medical problem,"두통 , 구역 , 의식장애 , 호흡장애 , 식욕 부진 , 구토 , 마비 ...","고산병은 낮은 지대에서 고도가 높은 해발 2,000~3,000m 이상의 고지대로 이...",급성산악병은 가벼운 두통과 숨이 답답한 증상으로 시작하여 대개 등산 후 1~6시간 ...,고지대로 올라가면 점차 공기 중 산소농도가 떨어져 동맥 혈액에 녹아든 산소가 줄고(...,"뇌, 허파",고산병은 증상을 보고 의심하는 것이 중요하다.,"1) 경한 급성 산악병\n- 등산을 중지하고, 낮은 고도로 내려가거나 혹은 같은 고...",http://www.snuh.org/health/nMedInfo/nView.do?c...


### 서울아산병원 데이터 크롤링 [(링크)](https://www.amc.seoul.kr/asan/healthinfo/disease/diseaseSubmain.do)

In [5]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time


def asan_crowling(disease, disease_en, idx, df):

    chrome_option = webdriver.ChromeOptions()
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_option)

    # 서울아산병원 홈페이지
    driver.get('https://www.amc.seoul.kr/asan/healthinfo/disease/diseaseSubmain.do')

    search_ipt = driver.find_element(By.XPATH, '//*[@id="regionSrc"]')
    search_ipt.send_keys(disease)
    search_ipt.send_keys(Keys.RETURN)
    time.sleep(1)
    
    
    
    if "없습니다" in driver.find_element(By.CLASS_NAME, 'listCont').text:
        search_ipt = driver.find_element(By.XPATH, '//*[@id="regionSrc"]')
        search_ipt.clear()
        search_ipt.send_keys(disease_en)
        search_ipt.send_keys(Keys.RETURN)
        time.sleep(1)


    results_list = driver.find_element(By.CLASS_NAME, 'descBox').find_elements(By.TAG_NAME, 'li')
    elem_list = []
    title_list = []
    syn_list = []
    for result in results_list:
        title_space = result.find_element(By.CLASS_NAME, 'contTitle')
        elem_list.append(title_space.find_element(By.TAG_NAME, 'a'))
        title_list.append(title_space.find_element(By.TAG_NAME, 'a').text.replace(" ", "").split('(')[0])
        title_space = driver.find_element(By.CLASS_NAME, 'contBox').find_elements(By.TAG_NAME, 'dd')

        if len(title_space) == 4:
            syn_list.append(result.find_element(By.CSS_SELECTOR, 'dd:nth-child(8)').text.replace(" ", '').split(','))
        else:
            syn_list.append([""])

    best = []
    for i in range(len(title_list)):
        title = title_list[i]
        syn = syn_list[i]            
            
        if disease.replace(" ", "") == title:
            best.insert(0, title_list.index(title))
        elif disease.replace(" ", "") in syn:
            best.insert(0, title_list.index(title))
        elif disease.replace(" ", "") in title:
            best.append(title_list.index(title))

        
    elem_list[best[0]].click()

    # for i in range(1, 11):
    #     if len(results_list) == 1:                          
    #         disease_syn = driver.find_element(By.XPATH, f'//*[@id="listForm"]/div/div/ul/li/div[2]/dl/dd[4]')
    #         disease_btn = driver.find_element(By.XPATH, f'//*[@id="listForm"]/div/div/ul/li/div[2]/strong/a')
    #     else:
    #         disease_syn = driver.find_element(By.XPATH, f'//*[@id="listForm"]/div/div/ul/li[{i}]/div[2]/dl/dd[4]')
    #         disease_btn = driver.find_element(By.XPATH, f'//*[@id="listForm"]/div/div/ul/li[{i}]/div[2]/strong/a')
            
    #     if disease.replace(" ", "") not in disease_btn.text.replace(" ", ""):
    #         if disease.replace(" ", "") in disease_syn.text.replace(" ", ""):
    #             disease_btn.click()
    #             time.sleep(1)
    #             break
    #     else:
    #         disease_btn.click()
    #         time.sleep(1)
    #         break
    
    xpath_dict = {
        "searched_name": '//*[@id="content"]/div[2]/div[1]/div[1]/ul/li/div[2]/strong',                   # 검색된 질환명
        "synonyms": '//*[@id="content"]/div[2]/div[1]/div[1]/ul/li/div[2]/dl/dd[4]',
        "symptom_keys": '//*[@id="content"]/div[2]/div[1]/div[1]/ul/li/div[2]/dl/dd[1]',              # 증상 키워드
        "define_text": '//*[@id="content"]/div[2]/div[1]/div[2]/dl/dd[1]/p[1]',        # 질환 정의
        "symptom_text": '//*[@id="content"]/div[2]/div[1]/div[2]/dl/dd[3]/p[1]',       # 증상
        "cause_text": '//*[@id="content"]/div[2]/div[1]/div[2]/dl/dd[2]/p',         # 원인
        "diagnosis_text": '//*[@id="content"]/div[2]/div[1]/div[2]/dl/dd[4]/p[1]',     # 진단
        "treat_text": '//*[@id="content"]/div[2]/div[1]/div[2]/dl/dd[5]/p',         # 치료
    }

    cols = ['질환명', '영문명', '동의어', '증상 키워드', '정의', '증상', '원인', '진단', '치료', 'URL']

    # 질환 세부 정보 수집
    err_list = []
    for info, xpath in xpath_dict.items():
        # 질환명은 따로 수집하여, 파싱을 통해 국문명과 영문명을 추출
        if info == 'searched_name':
            info = driver.find_element(By.XPATH, xpath).text
            kr_name = info.split('(')[0]
            en_name = info.split('(')[1].replace(')','')
            df.loc[idx, '질환명'] = kr_name
            df.loc[idx, '영문명'] = en_name
        elif info == "synonyms":
            info = driver.find_element(By.XPATH, xpath).text.split(',')
            df.loc[idx, '동의어'] = info
        elif info == "symtom_keys":
            info = driver.find_element(By.XPATH, xpath).text
            symtoms = info.find_elements(By.TAG_NAME, 'a')
            df.loc[idx, '증상 키워드'] = symtoms
        else: 
            # 예외 처리
            try:
                info = driver.find_element(By.XPATH, xpath).text
            except:
                err_list.append(info)
                info = None
                continue
            df.loc[idx, cols[list(xpath_dict.values()).index(xpath)+1]] = info

    df.loc[idx, 'URL'] = driver.current_url

    if len(err_list) != 0:
        print(f"Error: '{disease}'의 '{err_list}' 정보 수집 실패")
    else:
        print(f"'{disease}'의 정보 수집 완료")
    driver.close()

In [6]:
import pandas as pd
snu_df = pd.read_csv('snuh_crowling.csv')
disease_kr = snu_df['질환명'].tolist()
disease_en = snu_df['영문명'].tolist()
disease_dict = dict.fromkeys(disease_kr)
for i in range(len(disease_kr)):
    disease_dict[disease_kr[i]] = disease_en[i]
    
# 질환 20개 리스트
diseases = disease_kr

asan_df = pd.DataFrame(columns=['질환명', '영문명', '동의어', '증상 키워드', '정의', '증상', '원인', '진단', '치료', "URL"])

for idx in range(len(diseases)):
    disease = diseases[idx]
    asan_crowling(disease, disease_dict[disease], idx, asan_df)
    # try:
    # except:
    #     print(f"Error: '{disease}'의 정보 수집 실패")
    #     continue

'과민성 대장 증후군'의 정보 수집 완료
'당뇨병'의 정보 수집 완료
'심부전'의 정보 수집 완료
'천식'의 정보 수집 완료
'급성 대장염'의 정보 수집 완료
'감기'의 정보 수집 완료
'급성 경막하 출혈'의 정보 수집 완료
'인플루엔자'의 정보 수집 완료
'긴장성 두통'의 정보 수집 완료
'고산병'의 정보 수집 완료
'아나필락시스'의 정보 수집 완료
'루게릭병'의 정보 수집 완료
'장폐색'의 정보 수집 완료
'아토피 피부염'의 정보 수집 완료
'대상포진'의 정보 수집 완료
'돌발성 난청'의 정보 수집 완료
'만성 비염'의 정보 수집 완료
'급성 신우신염'의 정보 수집 완료
'뇌전증'의 정보 수집 완료
'패혈증'의 정보 수집 완료


In [7]:
asan_df.to_csv('asan_crowling.csv', index=False, encoding='utf-8-sig')
asan_df

,질환명,영문명,동의어,증상 키워드,정의,증상,원인,진단,치료,URL
0,자극성 장 증후군,Irritable bowel syndrome,"[과민성대장, 과민성대장증후군, 과민성장증후군, 과민성창자, 과민성창자증후군, 자극...","복부 통증, 설사, 체중감소, 점액변, 변비",자극성 장 증후군은 특별한 기질적 원인이 확인되지 않는 상태에서 만성적으로 지속되는...,대장은 정상적으로 하루 1회 정도 배변을 유도하도록 움직입니다. 그러나 대장이 과민...,아직까지 자극성 장 증후군의 원인은 명확히 밝혀지지 않았습니다. 유전적 가족적 요인...,자극성 장 증후군은 구조적 이상으로 설명할 수 없으므로 특정 검사 방법으로 진단하기...,자극성 장 증후군의 원인이 되는 심리적 불안과 갈등을 제거하는 것이 가장 중요합니다...,https://www.amc.seoul.kr/asan/healthinfo/disea...
1,당뇨병,Diabetes mellitus,[당뇨],"고혈당, 소양감, 체중감소, 피로감, 시야장애, 다뇨, 저림, 다음, 다식",당뇨병은 소변으로 포도당이 배출된다고 하여 이름 붙여진 병입니다. 정상인의 경우 소...,"당뇨병에 걸리면 소변으로 포도당이 빠져나가는데, 이때 수분을 같이 끌고 나가기 때문...","당뇨병의 발생에는 유전과 환경이 중요한 역할을 합니다. 즉, 당뇨병은 이것에 걸리기...",당뇨병은 혈당을 측정하여 진단합니다. 이때 혈당 검사란 손가락 끝에서 채혈을 하는 ...,"당뇨병의 치료 방법으로는 식사 요법, 운동 요법, 약물 치료가 있습니다. 가벼운 당...",https://www.amc.seoul.kr/asan/healthinfo/disea...
2,울혈성 심부전,Congestive heart failure,"[울혈성심부전증, 울혈심부전, 울혈심부전증]","빈맥, 심부전, 호흡곤란, 울혈성 심부전, 소변량 감소, 야뇨증, 두통, 불면증, 청색증",울혈성 심부전은 여러 원인으로 인해 심장이 신체 조직이나 기관에서 필요한 혈액(특히...,"울혈성 심부전의 일반적인 증상으로는 전반적으로 몸이 약해짐, 피곤함, 운동을 조금만...",나이가 들면 심장의 기능이 감소하여 울혈성 심부전이 발생할 위험성이 커집니다. 65...,"울혈성 심부전은 임상 증상, 신체 검사, 간단한 검사(심전도, 흉부 X-ray 촬영...",1. 약물 요법,https://www.amc.seoul.kr/asan/healthinfo/disea...
3,천식,Asthma,"[소아천식, 운동유발성천식, 직업성천식]","천명음, 가래, 가슴 답답, 기침, 호흡곤란",천식은 알레르기 염증에 의해 기관지가 반복적으로 좁아지는 만성 호흡기 질환입니다. ...,기관지 천식은 유전적 요인과 환경적인 요인이 함께 상호작용하여 나타납니다.,1. 대기 알레르겐,"1. 문진과 진찰\n기침, 천명, 호흡 곤란 등의 증상이 있습니다. 특히 야간이나 ...",1. 환경 관리\n원인이 되는 대기 알레르겐과 비특이적 자극원을 차단합니다.,https://www.amc.seoul.kr/asan/healthinfo/disea...
4,대장염,Colitis,"[급성 대장염, 대장질환, 만성 대장염]","설사, 점액변, 배변습관의 변화",약 1.5m 길이의 대장은 크게 결장과 직장으로 구분됩니다. 결장은 소장과 연결된 ...,"대장염 중에서도 각각의 질환에 따라 증상이 다릅니다. 다만 복통, 설사 등이 공통적...",대장염은 방대한 범위의 질환을 포함하는 개념입니다. 따라서 대장염의 원인은 구체적인...,"대장염을 진단하기 위해 혈액 검사, 대변 검사, 복부 컴퓨터 촬영, 대장 내시경 검...",최근 서구화된 식습관으로 인해 대장암을 비롯한 대장 건강 문제가 증가하고 있습니다....,https://www.amc.seoul.kr/asan/healthinfo/disea...
5,상기도 감염,Upper respiratory infections,"[common cold, 감기]","열, 기침, 피로감, 목의 통증, 두통, 코막힘, 재채기, 콧물","상기도 감염은 코, 인두, 후두, 기관 등 상기도의 감염성 염증 질환을 의미합니다....",콧물과 코막힘이 계속되면 구강으로 호흡을 하게 되어 목이 마르고 붓습니다. 편도 쪽...,상기도 감염(감기)은 일반적으로 바이러스 때문에 발생하는 경우가 많습니다. 세균 감...,단순한 상기도 감염은 보통 임상적으로 진단하고 대증 요법으로 치료합니다. 그러나 고...,상기도 감염은 보통 증상에 따른 대증 요법으로 약물 치료를 시행합니다. 일부 세균에...,https://www.amc.seoul.kr/asan/healthinfo/disea...
6,외상성 경막하 출혈,Traumatic subdural hemorrhage,"[subdural hemorrhage, 경막하 출혈, 경막하혈종, 급성 경막하 출혈]","호흡곤란, 구토, 혼수, 두통, 반신마비",외상성 경막하 출혈이란 교통사고와 같은 외부 충격으로 인해 뇌를 둘러싸고 있는 경막...,"외상성 경막하 출혈의 가장 흔한 증상은 두통, 한쪽의 위약감, 경련, 시야 장애, ...",외상성 경막하 출혈은 대부분 머리에 대한 직접적인 충격으로 뇌와 경막 사이의 혈관이...,"외상성 경막하 출혈을 진단하기 위해 혈압과 맥박을 측정하고, 시야와 안구의 빛 반응...",검사 결과상 출혈량이 적고 환자의 상태가 양호한 경우 수술적 치료 없이 경과를 관찰...,https://www.amc.seoul.kr/asan/healthinfo/disea...
7,인플루엔자,Influenza,[독감],"오한, 열, 설사, 근육통, 구토, 목의 통증, 두통",인플루엔자는 감기 증세를 일으키는 바이러스 중 인플루엔자(influenza) 바이러...,"독감의 증세는 일반적인 감기보다 심합니다. 피로감이 동반된 고열이 생기고, 심한 두...","바이러스 내에서 전자 돌연변이가 지속적으로 생기게 되면, 면역력이 없는 항원을 가진...",독감은 지역사회에 독감이 얼마나 유행하는지에 따라서 독감 유사 증상(influenz...,독감을 치유하기 위해서는 충분한 휴식과 수면 등 안정을 취하는 것이 중요합니다. 합...,https://www.amc.seoul.kr/asan/healthinfo/disea...
8,근육 긴장성 두통,Tension-type headache,[긴장성두통],"압통, 두통",근육 긴장성 두통은 대부분의 사람들이 경험할 정도로 가장 흔한 두통입니다. 일반적으...,"근육 긴장성 두통 환자가 호소하는 통증은 일반적으로 박동성이 아닌 압박감, 조이는 ...","지속적인 근 수축의 결과로 두통이 발생합니다. 자신도 모르게 이를 깨물거나, 힘을 ...",삽화성 긴장성 두통 진단 기준(국제두통학회),근육 긴장성 두통은 진통제에 잘 반응합니다. 저빈도 삽화성(episodic) 긴장성...,https://www.amc.seoul.kr/asan/healthinfo/disea...
9,고산병,Mountain sickness,"[altitude sickness, 산악병]","탈수, 식욕부진, 구토, 의식 변화, 두통, 수면장애, 어지러움, 말초부종",고산병은 산의 고도가 높아짐에 따라 저산소증이 생길 때 이를 보상하기 위한 신체의 ...,"산의 오르막(경사) 정도나 산의 높이, 고산지대에서의 신체활동 정도, 개인의 감수성...",높은 곳은 낮은 곳보다 산소가 부족하여 호흡 시 산소의 흡입이 부족해집니다. 이를 ...,안정을 취하고 있어도 맥박이 110회/분 이상 뛰고 호흡은 20회/분 이상 빠르게 ...,① 산을 내려오는 게 가장 좋습니다.\n② 산소를 사용할 수 있는 경우 산소를 분당...,https://www.amc.seoul.kr/asan/healthinfo/disea...
